In [1]:
import pypylon.pylon as py
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [2]:
op_name = "output.avi"
fw = 1280
fh = 720
fps = 200
fourcc = cv2.VideoWriter_fourcc(*'XVID')

In [3]:
os.environ["PYLON_CAMEMU"] = "1"

Two steps to do 

1. Create an instance of transport layer which is responsible for detecting and managing connected cameras. It is required to communicate with the camera hardware.
2. Create a camera object for the camera that you are interested in and wrap it with InstantCamera for ease of handling.

In [4]:
tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
cam = py.InstantCamera(tlf.CreateDevice(devices[0]))

In [5]:
cam.Width

<pypylon.genicam.IInteger; proxy of <Swig Object of type 'GENAPI_NAMESPACE::IInteger *' at 0x00000153FE49C240> >

Initialize the connection to the Basler camera. This is a required step before we can capture or stream video. Camera configuration can be set once it is opened.

In [6]:
cam.Open()

Configuring the camera

In [7]:
cam.Width.Value = fw
cam.Height.Value = fh
cam.AcquisitionFrameRate.Value = fps

There are different strategies to use for camera grabbing. 
`LatestImageOnly` might be useful for streaming while `OneByOne` might be useful for storing to disk

In [8]:
cam.StartGrabbing(py.GrabStrategy_OneByOne)

In [9]:
video_writer = cv2.VideoWriter(op_name, fourcc, fps, (fw, fh))

In [10]:
frame_count = 0
max_frames = 400

while cam.IsGrabbing() and frame_count < max_frames:
    grab_result = cam.RetrieveResult(5000, py.TimeoutHandling_ThrowException)

    if grab_result.GrabSucceeded():
        img = grab_result.Array
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) 

        video_writer.write(img)
        frame_count += 1

        # cv2.imshow("Live Feed", img)
        # if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to stop recording
        #     break

    grab_result.Release()

In [11]:
cam.StopGrabbing()

In [12]:
video_writer.release()

In [13]:
cv2.destroyAllWindows()

In [14]:
cam.Close()